# Preprocess ODK data to organized tables

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import os
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import DATA_DIR, GCP_PROJ_ID
from src.biomass_inventory import (
    extract_trees,
    extract_stumps,
    extract_dead_trees_class1,
    extract_dead_trees_class2s,
    extract_dead_trees_class2t,
    extract_ldw_with_hollow,
    extract_ldw_wo_hollow,
)

In [3]:
# Variables
URL = "https://api.ona.io/api/v1/data/763932.csv"
FILE_RAW = DATA_DIR / "csv" / "biomass_inventory_raw.csv"
CARBON_POOLS_OUTDIR = DATA_DIR / "csv" / "carbon_pools"
NESTS = [2, 3, 4]

# BigQuery Variables
DATASET_ID = "biomass_inventory"
IF_EXISTS = "replace"

In [4]:
# Create output directory
CARBON_POOLS_OUTDIR.mkdir(parents=True, exist_ok=True)

# Get Data from ONA

In [5]:
column_types = {
    col: str
    for col in (
        28,
        399,
        400,
        407,
        408,
        415,
        416,
        845,
        846,
        853,
        854,
        861,
        862,
        869,
        870,
        877,
        878,
        885,
        886,
        893,
        894,
        901,
        902,
        909,
        910,
        1179,
        1180,
        1187,
        1188,
        1195,
        1196,
        1203,
        1204,
        1211,
        1212,
        1219,
        1220,
        1286,
        1337,
        1342,
        1347,
        1352,
        1357,
        1362,
        1378,
        1392,
    )
}

In [6]:
if FILE_RAW.exists():
    data = pd.read_csv(FILE_RAW, dtype=column_types)
else:
    urllib.request.urlretrieve(URL, FILE_RAW)
    data = pd.read_csv(FILE_RAW, dtype=column_types)

## Add a unique ID

In [7]:
# Create a new column with "1" for Primary and "2" for Backup
data["plot_type_short"] = data["plot_info/plot_type"].apply(
    lambda x: "1" if x == "Primary" else "2"
)

# Extract subplot letters (assuming they are included in the 'plot_info.sub_plot' column)
data["subplot_letter"] = data["plot_info/sub_plot"].str.replace("sub_plot", "")

# Create the unique ID by concatenating the specified columns
data["unique_id"] = (
    data["plot_info/plot_code_nmbr"].astype(str)
    + data["subplot_letter"]
    + data["plot_type_short"]
)

# Extract Plot info

In [8]:
plot_info_cols = [
    "unique_id",
    "plot_info/data_recorder",
    "plot_info/team_no",
    "plot_info/plot_code_nmbr",
    "plot_info/plot_type",
    "plot_info/sub_plot",
    "plot_info/yes_no",
    "plot_shift/sub_plot_shift",
    "plot_GPS/GPS_waypt",
    "plot_GPS/GPS_id",
    "plot_GPS/GPS",
    "plot_GPS/_GPS_latitude",
    "plot_GPS/_GPS_longitude",
    "plot_GPS/_GPS_altitude",
    "plot_GPS/_GPS_precision",
    "plot_GPS/photo",
    "access/access_reason/slope",
    "access/access_reason/danger",
    "access/access_reason/distance",
    "access/access_reason/water",
    "access/access_reason/prohibited",
    "access/access_reason/other",
    "access/manual_reason",
    "lc_data/lc_type",
    "lc_class/lc_class",
    "lc_class/lc_class_other",
    "disturbance/disturbance_yesno",
    "disturbance_data/disturbance_type",
    "disturbance_class/disturbance_class",
    "slope/slope",
    "canopy/avg_height",
    "canopy/can_cov",
    "sapling_data/count_saplings",
    "ntv_data/litter_data/litter_bag_weight",
    "ntv_data/litter_data/litter_sample_weight",
    "ntv_data/ntv_bag_weight",
    "ntv_data/ntv_sample_weight",
]

In [9]:
plot_info = data[plot_info_cols].copy()

In [10]:
# rename columns
plot_info_cols = {
    "plot_info/data_recorder": "data_recorder",
    "plot_info/team_no": "team_no",
    "plot_info/plot_code_nmbr": "plot_code_nmbr",
    "plot_info/plot_type": "plot_type",
    "plot_info/sub_plot": "sub_plot",
    "plot_info/yes_no": "yes_no",
    "plot_shift/sub_plot_shift": "sub_plot_shift",
    "plot_GPS/GPS_waypt": "GPS_waypt",
    "plot_GPS/GPS_id": "GPS_id",
    "plot_GPS/GPS": "GPS",
    "plot_GPS/_GPS_latitude": "GPS_latitude",
    "plot_GPS/_GPS_longitude": "GPS_longitude",
    "plot_GPS/_GPS_altitude": "GPS_altitude",
    "plot_GPS/_GPS_precision": "GPS_precision",
    "plot_GPS/photo": "photo",
    "access/access_reason/slope": "access_reason_slope",
    "access/access_reason/danger": "access_reason_danger",
    "access/access_reason/distance": "access_reason_distance",
    "access/access_reason/water": "access_reason_water",
    "access/access_reason/prohibited": "access_reason_prohibited",
    "access/access_reason/other": "access_reason_other",
    "access/manual_reason": "manual_reason",
    "lc_data/lc_type": "lc_type",
    "lc_class/lc_class": "lc_class",
    "lc_class/lc_class_other": "lc_class_other",
    "disturbance/disturbance_yesno": "disturbance_yesno",
    "disturbance_data/disturbance_type": "disturbance_type",
    "disturbance_class/disturbance_class": "disturbance_class",
    "slope/slope": "slope",
    "canopy/avg_height": "canopy_avg_height",
    "canopy/can_cov": "canopy_cover",
    "sapling_data/count_saplings": "count_saplings",
    "ntv_data/litter_data/litter_bag_weight": "litter_bag_weight",
    "ntv_data/litter_data/litter_sample_weight": "litter_sample_weight",
    "ntv_data/ntv_bag_weight": "ntv_bag_weight",
    "ntv_data/ntv_sample_weight": "ntv_sample_weight",
}

In [11]:
plot_info.rename(columns=plot_info_cols, inplace=True)

### Set correct data types

In [12]:
column_types = {
    "unique_id": str,
    "data_recorder": str,
    "team_no": int,
    "plot_code_nmbr": int,
    "plot_type": str,
    "sub_plot": str,
    "yes_no": str,
    "sub_plot_shift": str,
    "GPS": str,
    "photo": str,
    "access_reason_slope": str,
    "access_reason_danger": str,
    "access_reason_distance": str,
    "access_reason_water": str,
    "access_reason_prohibited": str,
    "access_reason_other": str,
    "manual_reason": str,
    "lc_type": str,
    "lc_class": str,
    "lc_class_other": str,
    "disturbance_yesno": str,
    "disturbance_type": str,
    "disturbance_class": str,
}

In [13]:
plot_info = plot_info.astype(column_types)

### Compress access reasons to one column

In [14]:
plot_info["access_reason"] = np.nan
plot_info["access_reason"] = plot_info["access_reason"].astype(str)
for index, row in plot_info.iterrows():
    if row["access_reason_slope"] == "True":
        plot_info.loc[index, "access_reason"] = "slope"
    elif row["access_reason_danger"] == "True":
        plot_info.loc[index, "access_reason"] = "danger"
    elif row["access_reason_distance"] == "True":
        plot_info.loc[index, "access_reason"] = "distance"
    elif row["access_reason_water"] == "True":
        plot_info.loc[index, "access_reason"] = "water"
    elif row["access_reason_prohibited"] == "True":
        plot_info.loc[index, "access_reason"] = "prohibited"
    elif row["access_reason_other"] == "True":
        plot_info.loc[index, "access_reason"] = row["manual_reason"]

In [15]:
# Categorize manual reasons
plot_info.loc[plot_info.access_reason == "90 degree slope ", "access_reason"] = "slope"
plot_info.loc[
    plot_info.access_reason == "Slippery due to rainfall and sharp stones..too risky.",
    "access_reason",
] = "danger"
plot_info.loc[
    plot_info.access_reason == "Creek plot and slope 90 degree", "access_reason"
] = "slope"
plot_info.loc[
    plot_info.access_reason == "Near creek 90 degree slope", "access_reason"
] = "slope"

In [16]:
# drop access_reason columns
plot_info.drop(
    columns=[
        "access_reason_slope",
        "access_reason_danger",
        "access_reason_distance",
        "access_reason_water",
        "access_reason_prohibited",
        "access_reason_other",
    ],
    inplace=True,
)

In [17]:
plot_info.access_reason.value_counts()

access_reason
nan       634
slope      31
danger      9
Name: count, dtype: int64

In [18]:
plot_info.info(), plot_info.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_id             674 non-null    object 
 1   data_recorder         674 non-null    object 
 2   team_no               674 non-null    int64  
 3   plot_code_nmbr        674 non-null    int64  
 4   plot_type             674 non-null    object 
 5   sub_plot              674 non-null    object 
 6   yes_no                674 non-null    object 
 7   sub_plot_shift        674 non-null    object 
 8   GPS_waypt             634 non-null    float64
 9   GPS_id                634 non-null    float64
 10  GPS                   674 non-null    object 
 11  GPS_latitude          577 non-null    float64
 12  GPS_longitude         577 non-null    float64
 13  GPS_altitude          577 non-null    float64
 14  GPS_precision         577 non-null    float64
 15  photo                 6

(None,
   unique_id data_recorder  team_no  plot_code_nmbr plot_type   sub_plot  \
 0     308D2         Steve        1             308   primary  sub_plotD   
 1     308A2         Steve        1             308   primary  sub_plotA   
 
   yes_no sub_plot_shift  GPS_waypt  GPS_id  ... disturbance_class  slope  \
 0    yes       no_shift        7.0     1.0  ...               nan   23.0   
 1    yes       no_shift        8.0     1.0  ...               nan   13.0   
 
    canopy_avg_height  canopy_cover  count_saplings litter_bag_weight  \
 0               12.0           3.0             2.0              70.0   
 1                8.0           4.0             NaN              50.0   
 
   litter_sample_weight ntv_bag_weight ntv_sample_weight access_reason  
 0                770.0           70.0             870.0           nan  
 1                260.0           50.0             110.0           nan  
 
 [2 rows x 32 columns])

## Export data and upload to BQ

In [19]:
# Export CSV
if len(plot_info) != 0:
    plot_info.to_csv(CARBON_POOLS_OUTDIR / "plot_info.csv", index=False)

In [20]:
# Upload to BQ
if len(plot_info) != 0:
    pandas_gbq.to_gbq(
        plot_info,
        f"{DATASET_ID}.plot_info",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 6626.07it/s]


# Extract info per carbon pool

# Living Trees

In [ ]:
trees = extract_trees(data, NESTS)

In [ ]:
trees.info(), trees.head(2)

In [ ]:
trees.describe()

## Export data and upload to BQ

In [ ]:
# Export to CSV
trees.to_csv(CARBON_POOLS_OUTDIR / "trees.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(
    trees, f"{DATASET_ID}.trees", project_id=GCP_PROJ_ID, if_exists=IF_EXISTS
)

# Tree Stumps

[delete when fixed] Note: removed `'biomass_per_kg_tree': [biomass_per_kg_tree],`. In the original code there was a placeholder column created, this can be added later in the process when biomass per tree is actually calculated

In [ ]:
stumps = extract_stumps(data, NESTS)

In [ ]:
stumps.info(), stumps.head(2)

In [ ]:
stumps.describe()

## Export data and upload to BQ

In [ ]:
# Export to CSV
stumps.to_csv(CARBON_POOLS_OUTDIR / "stumps.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(
    stumps, f"{DATASET_ID}.stumps", project_id=GCP_PROJ_ID, if_exists=IF_EXISTS
)

# Dead Trees: Class 1

In [ ]:
dead_trees_c1 = extract_dead_trees_class1(data, NESTS)

In [ ]:
dead_trees_c1.info(), dead_trees_c1.head(2)

In [ ]:
dead_trees_c1.describe()

## Export data and upload to BQ

In [ ]:
dead_trees_c1.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class1.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(
    dead_trees_c1,
    f"{DATASET_ID}.dead_trees_c1",
    project_id=GCP_PROJ_ID,
    if_exists=IF_EXISTS,
)

# Dead Trees: Class 2 - short

In [ ]:
dead_trees_c2s = extract_dead_trees_class2s(data, NESTS)

In [ ]:
dead_trees_c2s.info(), dead_trees_c2s.head(2)

## Export data and upload to BQ

In [ ]:
# Export CSV
if len(dead_trees_c2s) != 0:
    dead_trees_c2s.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class2.csv", index=False)

In [ ]:
# Upload to BQ
if len(dead_trees_c2s) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c2s,
        f"{DATASET_ID}.dead_trees_c2s",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

# Dead Trees: Class 2 - Tall

[delete when fixed] Note: This is still class 2 but tall trees. rename variables and functions accordingly

In [ ]:
dead_trees_c2t = extract_dead_trees_class2t(data, NESTS)

In [ ]:
dead_trees_c2t.info(), dead_trees_c2t.head(2)

## Export data and upload to BQ

In [ ]:
# Export CSV
if len(dead_trees_c2t) != 0:
    dead_trees_c2t.to_csv(
        CARBON_POOLS_OUTDIR / "dead_trees_class2_tall.csv", index=False
    )

In [ ]:
# Upload to BQ
if len(dead_trees_c2t) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c2t,
        f"{DATASET_ID}.dead_trees_c2t",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

# Lying Deadwood: Hollow

In [ ]:
ldw_hollow = extract_ldw_with_hollow(data)

In [ ]:
ldw_hollow.info(), ldw_hollow.head(2)

## Export data and upload to BQ

In [ ]:
# Export CSV
if len(ldw_hollow) != 0:
    ldw_hollow.to_csv(CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv", index=False)

In [ ]:
# Upload to BQ
if len(ldw_hollow) != 0:
    pandas_gbq.to_gbq(
        ldw_hollow,
        f"{DATASET_ID}.lying_deadwood_hollow",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

# Lying Deadwood without hollow

In [ ]:
ldw_wo_hollow = extract_ldw_wo_hollow(data)

In [ ]:
ldw_wo_hollow.info(), ldw_wo_hollow.head(2)

## Export data and upload to BQ

In [ ]:
# Export CSV
if len(ldw_wo_hollow) != 0:
    ldw_wo_hollow.to_csv(
        CARBON_POOLS_OUTDIR / "lying_deadwood_wo_hollow.csv", index=False
    )

In [ ]:
# Upload to BQ
if len(ldw_wo_hollow) != 0:
    pandas_gbq.to_gbq(
        ldw_wo_hollow,
        f"{DATASET_ID}.lying_deadwood_wo_hollow",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )